In [4]:
#pip install uform

  Obtaining dependency information for uform from https://files.pythonhosted.org/packages/83/83/8c4ea07a3fe4e776482a79120167bd0711fe00f8c1e118d1601025e58159/uform-1.1.1-py3-none-any.whl.metadata
  Using cached uform-1.1.1-py3-none-any.whl.metadata (19 kB)
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/e7/45/419aa0b37254f1fd62b45bb63836066c5eb81e37d70940e0491e95167eed/torchvision-0.17.1-cp311-cp311-win_amd64.whl.metadata
  Using cached torchvision-0.17.1-cp311-cp311-win_amd64.whl.metadata (6.6 kB)
Using cached uform-1.1.1-py3-none-any.whl (22 kB)
Using cached torchvision-0.17.1-cp311-cp311-win_amd64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from uform.gen_model import VLMForCausalLM, VLMProcessor
import requests
from PIL import Image
import pandas as pd
from tqdm import tqdm
import torch

# Initialize the model and processor
model = VLMForCausalLM.from_pretrained("unum-cloud/uform-gen")
processor = VLMProcessor.from_pretrained("unum-cloud/uform-gen")

# Read the CSV file
csv_file = 'sampled_products_10_each_img_front.csv'  # Adjust to your file path
df = pd.read_csv(csv_file)

# Column name containing the URLs
url_column = 'ItemDocumentValue'

# List to store generated captions
captions = []

# Define your prompt
prompt = "[cap] Generate a thorough product description that appeals to customers."

# Process each image URL with a progress bar
for url in tqdm(df[url_column], desc="Processing images"):
    try:
        # Load and process the image
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status()
        image = Image.open(response.raw)

        # Prepare inputs
        inputs = processor(texts=[prompt], images=[image], return_tensors="pt")

        # Generate output
        with torch.inference_mode():
            output = model.generate(
                **inputs,
                do_sample=False,
                use_cache=True,
                max_new_tokens=128,
                eos_token_id=32001,
                pad_token_id=processor.tokenizer.pad_token_id
            )

        # Decode the text
        prompt_len = inputs["input_ids"].shape[1]
        decoded_text = processor.batch_decode(output[:, prompt_len:])[0]

        # Append the caption to the list
        captions.append(decoded_text)

    except Exception as e:
        # Handle any exceptions
        captions.append(f"Error processing image: {e}")

# Create a new DataFrame with only the desired columns and captions
output_df = pd.DataFrame({
    'ItemSku': df['ItemSku'],
    'ItemDocumentValue': df['ItemDocumentValue'],
    'generated_captions': captions
})

# Save the new DataFrame to a CSV file
output_df.to_csv('Uform-output_captions80.csv', index=False)

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

C:\Users\Jensen R\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jensen R\.cache\huggingface\hub\models--princeton-nlp--Sheared-LLaMA-1.3B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Processing images:  44%|███████████████████████████▌                                   | 35/80 [10:02<11:53, 15.85s/it]C:\Users\Jensen R\anaconda3\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Processing images: 100%|███████████████████████████████████████████████████████████████| 80/80 [24:53<00:00, 18.67s/it]
